# Night Sky Ransomware

Sample (packed): `8c1a72991fb04dc3a8cf89605fb85150ef0e742472a0c58b8fa942a1f04877b0`

Sample (dumped/unpacked): [`1fca1cd04992e0fcaa714d9dfa97323d81d7e3d43a024ec37d1c7a2767a17577`](https://malshare.com/sample.php?action=detail&hash=1fca1cd04992e0fcaa714d9dfa97323d81d7e3d43a024ec37d1c7a2767a17577)

References:
- [First seen from MalwareHunterTream](https://mobile.twitter.com/malwrhunterteam/status/1477381209147723788)
- [Sample VT](https://www.virustotal.com/gui/file/8c1a72991fb04dc3a8cf89605fb85150ef0e742472a0c58b8fa942a1f04877b0)
- [Sample Malshare](https://malshare.com/sample.php?action=detail&hash=9608c8b6c8d80fdc67b99edd3c53d3d2)
- [Malware Bazaar](https://bazaar.abuse.ch/sample/8c1a72991fb04dc3a8cf89605fb85150ef0e742472a0c58b8fa942a1f04877b0/)
- [id-ransomware](https://id-ransomware.blogspot.com/2022/01/nightsky-ransomware.html)


## Dynamic Analysis
- Drops `hta` ransom note in every directory (`NightSkyReadMe.hta`)
- Uses `.nightsky` file extension for encrypted files

## Unpacking
The sample is packed with some version of VMProtect, likely a newer version (3?) but I'm unsure. It seems to have some anti-debug that actually works against [ScyllaHide](https://www.patreon.com/posts/installing-to-57091901) but luckily it will run with a debugger attached. [MrExodia](https://twitter.com/mrexodia) confirmed that the entrypoint was virutalized but we discovered that the main funcationlity was not. This allowed us to break in the main functionality and use [vmpdump](https://github.com/0xnobody/vmpdump) to dump the payload and resolve the imports. 

## Static Analysis
The code is strightforward but it contains a statically linked [mbedtls](https://github.com/ARMmbed/mbedtls) library. To assist with reverse engineering we used lumina to pull down some function names for the mbedtls functions, and we compiled version `3.0.0` of mbedtls and imported the `.pdb` file to give us access to the library structs in IDA. 

## OSINT

It seems as though this might be a updated version of Rook Ransomware (January 8, 2022).

[Tweet from @vinopaljiri](https://twitter.com/vinopaljiri/status/1480059715392622597)

> Many researchers are spreading something what is not true.. Please repair your analytical conclusions.
#Rook #ransomware is basically previous version of new #NightSky - delivered VMProtected (will be shared later).
Facts about #Rook in pictures!!!

<img src="https://pbs.twimg.com/media/FIo6BoiXEAICKY8?format=png&name=large" 
alt="IMAGE ALT TEXT HERE" border="10" />

<img src="https://pbs.twimg.com/media/FIo6DG0WYAIDabX?format=png&name=large" 
alt="IMAGE ALT TEXT HERE" border="10" />

<img src="https://pbs.twimg.com/media/FIo6D21WYAQMC3D?format=jpg&name=large" 
alt="IMAGE ALT TEXT HERE" border="10" />

<img src="https://pbs.twimg.com/media/FIo6EzrWQAEymZh?format=png&name=medium" 
alt="IMAGE ALT TEXT HERE" border="10" />


## Dynamically Extracted IOCs

RSA Public Keys
```
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAwetDt+9kp5JJGCb3YrqH
48g0rxFIaj5/NjMBvxtpa+7nO/lS0FQXxWJO78dTT6xW/UgVLPK4MvbGeIj17aQF
SqGHbRxTeoPrHufp4sM4J2IQYLc6YLYZMS6XT02rHOjumBJpEKyROQ+df5KU/06o
Rrhljc0Qco+qW8q/xYJQ9VFa87IJM6WM3wsydHVDDeGuWi4/PMUT4/GAa8/WMUYW
9Ebw7/hXd/aNX5LykeonN+nkJfbj1fZNTU81tc8Kx4rykLvMVE1H3AaT5ssCBt7p
AFkLLjplOXz3XmhH+J5vm5Ifi7T85j4D6O03qocO2gwezIikCDU2YAOOpJzkb5Ab
+wIDAQAB
-----END PUBLIC KEY-----
```

```
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAnw/AqzVYc7H6A34K3MQ6
J8nJeaCSegiSOIjhxveQeocKp6TzvL6W3usuMbJKGfL9b19KkiYYyK0WAJ/gsIpq
DH7xLCeOq8Lttp07PJmpyOLzzSyuSRKY3E7uSATXMzvJq4OuIkAcwWdce8OnQfS0
17VxNISQU6XQMjt81sqHTzvmg3DaeoqGb1oISWyDoRMFd5Akt04Iw//aNIoGIBvh
e/HP71LQpbyzb0RY0HBQuk3zlIDK6J5Hvr+Bkd4Tr5oh/oBya6RX7gW5t1FkY42c
/maunB4PiPgHiPdHtJ1LQE+EtSLWU4ZCc0finaLBh3ooySIcsLP9ehlWds1h1ZWN
2QIDAQAB
-----END PUBLIC KEY-----
```

**TODO: Why are there two RSA keys?**


### Ransom Note IOCs

```
Contact information 
Web Chat: 
You can use the username and password provided by us to login to the chat room to communicate with us.
URL:https://contact.nightsky.cyou
username:user-kdwckin
password:kdjfikerjijJHDUFIGjk3289
Email: 
You can contact us by email.
EMAIL ADDRESS:akij@nightsky.cyou

--------------------------------------------------------------------------------

Data release website 
Where we use to disclose the data of customers who do not pay 
http://gg5ryfgogainisskdvh4y373ap3b2mxafcibeh2lvq5x7fx76ygcosad.onion 
--------------------------------------------------------------------------------

Remark 
How to access dark web sites:https://www.youtube.com/watch?v=NpXEQHDOA5o 
```

## Mutex
The mutex `tset123155465463213` is used to prevent multiple cuncurrent versions of the ransomware. This might be a nice vaccine?


## Encryption

**TODO**